In [1]:
import uiautomator2 as ui2
import os
import pandas as pd
import time
import unittest
import json
import re
import csv

In [10]:
serial = '1333e1d50606'
d = ui2.connect_usb(serial)

In [4]:
serial = '1333e1d50606'
d = ui2.connect_usb(serial)
# 确认面板
if d(resourceId="co.leanremote.universalremotecontrol.remotecontrol:id/PowerOn1").exists:
    print(1, True)
    d(resourceId="co.leanremote.universalremotecontrol.remotecontrol:id/PowerOn1").click()
elif d(resourceId="co.leanremote.universalremotecontrol.remotecontrol:id/power").exists:
    print(2, True)
    d(resourceId="co.leanremote.universalremotecontrol.remotecontrol:id/power").click()
else:
    print(False)
d(resourceId="co.leanremote.universalremotecontrol.remotecontrol:id/working").click()

1 True


In [39]:
def level2redcode(code_list, frequency=38000, state=305):
    listValue = []
    listValue1 = []
    if isinstance(code_list, str):
        code_list = eval(code_list)
#     list1 = code_list[2:]  # 去掉频率和长度
    list1 = code_list  # 去掉频率和长度
    if not list1:
        return 38000, []
    for li in range(len(list1)):
        n = round(int(list1[li]) / state)
        if (n > 255):
            listValue1.append(0)
            listValue1.append(n >> 8 & 0xff)
            listValue1.append(n & 0xff)
        else:
            listValue1.append(n)
    q1 = round(int(frequency / 1000), 0)  # 要取小数将第一个int改为float，最后0改要求小数位
    if (q1 > 255):
        listValue.append(q1 >> 8 & 0xff)
        listValue.append(q1 & 0xff)
    else:
        listValue.append(q1)
        listValue.append(0)
    if (len(listValue1) > 255):
        listValue.append(len(listValue1) & 0xff)
        listValue.append(len(listValue1) >> 8 & 0xff)
    else:
        listValue.append(len(listValue1))
        listValue.append(0)
    listValue = listValue + listValue1
    return listValue

In [49]:
def get_redcode():
    fr = open("C:/Users/dell/Desktop/IRReader/IRReader2015/demo20190619-1/IRDemo/bin/Debug/err_code.txt", "r")
    levelcode = fr.read().split("\n")[-2]
    fr.close()
    redcode = level2redcode([int(i) for i in levelcode.split(" ")])
    return redcode

In [58]:
redcode=get_redcode()
os.remove("C:/Users/dell/Desktop/IRReader/IRReader2015/demo20190619-1/IRDemo/bin/Debug/err_code.txt")

In [59]:
print(redcode)

[38, 0, 188, 1, 116, 52, 17, 12, 17, 39, 17, 12, 17, 11, 17, 11, 17, 11, 17, 11, 17, 11, 17, 11, 17, 11, 17, 11, 17, 11, 17, 11, 17, 39, 17, 11, 17, 11, 17, 11, 17, 11, 17, 11, 17, 11, 17, 11, 17, 38, 18, 39, 18, 38, 18, 12, 17, 11, 17, 38, 18, 11, 17, 11, 17, 11, 17, 11, 17, 11, 17, 11, 17, 11, 17, 11, 17, 11, 17, 11, 17, 11, 17, 11, 17, 11, 17, 11, 17, 11, 17, 11, 17, 11, 17, 11, 17, 11, 17, 11, 17, 11, 17, 11, 17, 11, 17, 11, 17, 11, 17, 11, 17, 11, 17, 11, 17, 11, 17, 11, 17, 38, 18, 38, 17, 12, 17, 11, 17, 11, 17, 11, 17, 11, 17, 0, 1, 59, 115, 52, 18, 12, 17, 39, 17, 11, 17, 11, 17, 11, 17, 11, 17, 11, 17, 11, 17, 11, 17, 11, 17, 11, 17, 11, 17, 11, 17, 37, 18, 12, 17, 11, 17, 11, 17, 11, 17, 11, 17, 11, 17, 11, 17, 38, 17, 39, 17, 39, 18, 11, 17, 11, 17, 39, 17, 11, 17, 11, 17, 11, 17, 11, 17, 11, 17, 11, 17, 11, 17, 11, 17, 11, 17, 11, 17, 11, 17, 11, 17, 11, 17, 38, 17, 12, 17, 11, 17, 11, 17, 39, 17, 39, 17, 11, 17, 11, 17, 11, 17, 11, 17, 11, 17, 11, 17, 11, 17, 37, 18, 12, 

In [72]:
# 面板类型1,有制热,初始状态是 模式制冷, 温度16, 风速1
if d(resourceId="co.leanremote.universalremotecontrol.remotecontrol:id/Channel_Up").exists:
    print("面板类型1")
    rel_data = []
    # 加入关机键
    os.remove("C:/Users/dell/Desktop/IRReader/IRReader2015/demo20190619-1/IRDemo/bin/Debug/err_code.txt")
    d(resourceId="co.leanremote.universalremotecontrol.remotecontrol:id/PowerOff").click()
    time.sleep(1)
    redcode=str(get_redcode())
    rel_data.append(['close','cool','speed1','auto',27,redcode])
    # 其他开机状态
    temp_min = int(d(resourceId="co.leanremote.universalremotecontrol.remotecontrol:id/display").get_text().replace("°C", ""))
    for mode in ['cool', 'heat']:
        for speed in ['speed1', 'speed2', 'speed3', 'auto']:
             
            for temp in range(temp_min, 30):
                if temp == 16:
                    os.remove("C:/Users/dell/Desktop/IRReader/IRReader2015/demo20190619-1/IRDemo/bin/Debug/err_code.txt")
                    d(resourceId="co.leanremote.universalremotecontrol.remotecontrol:id/Volume_Down").click()
                    time.sleep(1)
                    redcode=str(get_redcode())
                    rel_data.append(['open',mode,speed,'auto',temp,redcode])
                
                os.remove("C:/Users/dell/Desktop/IRReader/IRReader2015/demo20190619-1/IRDemo/bin/Debug/err_code.txt")
                d(resourceId="co.leanremote.universalremotecontrol.remotecontrol:id/Volume_Up").click()
                time.sleep(1)
                redcode=str(get_redcode())
                rel_data.append(['open',mode,speed,'auto',temp+1,redcode])
                
                time.sleep(1)
            # 将温度重新退回16度
            for i in range(16):
                d(resourceId="co.leanremote.universalremotecontrol.remotecontrol:id/Volume_Down").click()
                time.sleep(1)
            if speed == 'auto':
                break
            d(resourceId="co.leanremote.universalremotecontrol.remotecontrol:id/Channel_Up").click()
            time.sleep(1)
        
        if mode == 'heat':
            break
        for i in range(4):
            d(resourceId="co.leanremote.universalremotecontrol.remotecontrol:id/Channel_Down").click()
            time.sleep(1)
        d(resourceId="co.leanremote.universalremotecontrol.remotecontrol:id/mode").click()
        time.sleep(1)
    df = pd.DataFrame(rel_data, columns=['state','mode','speed','direct','temperature','redcode'])
# 面板类型2
# 面板类型2,没有制热,初始状态是 没有显示模式和风速,并且风速不是状态码,独立的, 跳过,暂时不抓
else:
    pass


In [74]:
df = pd.DataFrame(rel_data, columns=['state','mode','speed','direct','temperature','redcode'])

In [75]:
df

,state,mode,speed,direct,temperature,redcode
0,close,cool,speed1,auto,27,"[38, 0, 188, 1, 115, 54, 17, 12, 16, 39, 17, 1..."
1,open,cool,speed1,auto,16,"[38, 0, 188, 1, 114, 54, 17, 12, 16, 40, 17, 1..."
2,open,cool,speed1,auto,17,"[38, 0, 188, 1, 113, 56, 15, 14, 14, 41, 16, 1..."
3,open,cool,speed1,auto,18,"[38, 0, 188, 1, 116, 52, 18, 12, 17, 38, 17, 1..."
4,open,cool,speed1,auto,19,"[38, 0, 188, 1, 116, 54, 17, 11, 17, 39, 17, 1..."
...,...,...,...,...,...,...
116,open,heat,auto,auto,26,"[38, 0, 188, 1, 116, 52, 18, 12, 17, 37, 18, 1..."
117,open,heat,auto,auto,27,"[38, 0, 188, 1, 116, 52, 18, 12, 17, 39, 17, 1..."
118,open,heat,auto,auto,28,"[38, 0, 188, 1, 116, 52, 18, 12, 17, 38, 18, 1..."
119,open,heat,auto,auto,29,"[38, 0, 188, 1, 116, 52, 18, 12, 17, 38, 18, 1..."


In [77]:

fr = open("空调品牌—红码状态.json", 'r')
rel = json.loads(fr.read())
fr.close()

In [78]:
rel

{'Philips': 0,
 'Panasonic': 1,
 'Samsung': 1,
 'Mitsubishi': 1,
 'Lloyd': 1,
 'Onida': 1,
 'Haier': 1,
 'Sanyo': 1,
 'LG': 1,
 'Kenwood': 1,
 'Gree': 1,
 'Aux': 1,
 'Daikin': 1,
 'Midea': 1,
 'Sharp': 1,
 'TCL': 1,
 'Toshiba': 1,
 'Bluestar': 1,
 'Bosch': 1,
 'Carrier': 1,
 'Daewoo': 1,
 'Electrolux': 1,
 'Friedrich': 1,
 'Fujitsu': 1,
 'GE': 1,
 'Godrej': 1,
 'Hisense': 1,
 'Hitachi': 1,
 'Hyundai': 1,
 'National': 1,
 'NEC': 1,
 'NEO': 1,
 'O-General': 1,
 'Olimpia-Splendid': 1,
 'Osaka': 1,
 'Pioneer': 1,
 'Premium': 1,
 'Sansui': 1,
 'Siemens': 1,
 'Singer': 1,
 'Trane': 1,
 'Uni-air': 1,
 'Videocon': 1,
 'Voltas': 1,
 'Westinghouse': 1,
 'Whirlpool': 1,
 'York': 1,
 'Acsom': 1,
 'Acson': 1,
 'Aermec': 1,
 'Agratto': 1,
 'Aidelong': 1,
 'Airfel': 1,
 'Airwave': 1,
 'Airwell': 1,
 'Aite': 1,
 'Alaska': 1,
 'Alliance': 1,
 'Almacom': 1,
 'Alpin': 1,
 'Amana': 0,
 'Amcor': 0,
 'Aoke': 1,
 'Apton': 1,
 'Artic King': 0,
 'Aucma': 0,
 'Azure': 1,
 'Ballu': 1,
 'Banshen': 1,
 'Baymak': 1

In [95]:
d(resourceId="com.android.systemui:id/back").click()
time.sleep(1)
d(resourceId="com.android.systemui:id/back").click()

In [97]:
d(resourceId="co.leanremote.universalremotecontrol.remotecontrol:id/PowerOff").exists

True

In [ ]:
serial = '1333e1d50606'
d = ui2.connect_usb(serial)
# 确认第二套面板
if d(resourceId="co.leanremote.universalremotecontrol.remotecontrol:id/PowerOn1").exists:
    print(1, True)
    d(resourceId="co.leanremote.universalremotecontrol.remotecontrol:id/PowerOn1").click()
elif d(resourceId="co.leanremote.universalremotecontrol.remotecontrol:id/power").exists:
    print(2, True)
    d(resourceId="co.leanremote.universalremotecontrol.remotecontrol:id/power").click()

d(resourceId="co.leanremote.universalremotecontrol.remotecontrol:id/notworking").click()

if d(resourceId="co.leanremote.universalremotecontrol.remotecontrol:id/PowerOn1").exists:
    print(1, True)
    d(resourceId="co.leanremote.universalremotecontrol.remotecontrol:id/PowerOn1").click()
elif d(resourceId="co.leanremote.universalremotecontrol.remotecontrol:id/power").exists:
    print(2, True)
    d(resourceId="co.leanremote.universalremotecontrol.remotecontrol:id/power").click()